<a href="https://colab.research.google.com/github/wndtrwhyn/ADSB-WINDA/blob/master/DBA_Lab_Test_Winda_Tri_Wahyuni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import math
import re
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)
pd.options.display.float_format = '{:.2f}'.format
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
data = pd.concat([pd.read_csv('/content/drive/My Drive/DBA Lab Test/DBALab-Test/Preprocessed/mHealth_subject1.csv',header=None)
                  ,pd.read_csv('/content/drive/My Drive/DBA Lab Test/DBALab-Test/Preprocessed/mHealth_subject2.csv',header=None)
                  ,pd.read_csv('/content/drive/My Drive/DBA Lab Test/DBALab-Test/Preprocessed/mHealth_subject3.csv',header=None)
                  ,pd.read_csv('/content/drive/My Drive/DBA Lab Test/DBALab-Test/Preprocessed/mHealth_subject4.csv',header=None)
                  ,pd.read_csv('/content/drive/My Drive/DBA Lab Test/DBALab-Test/Preprocessed/mHealth_subject5.csv',header=None)
                  ,pd.read_csv('/content/drive/My Drive/DBA Lab Test/DBALab-Test/Preprocessed/mHealth_subject6.csv',header=None)
                  ,pd.read_csv('/content/drive/My Drive/DBA Lab Test/DBALab-Test/Preprocessed/mHealth_subject7.csv',header=None)
                  ,pd.read_csv('/content/drive/My Drive/DBA Lab Test/DBALab-Test/Preprocessed/mHealth_subject8.csv',header=None)
                  ,pd.read_csv('/content/drive/My Drive/DBA Lab Test/DBALab-Test/Preprocessed/mHealth_subject9.csv',header=None)
                  ,pd.read_csv('/content/drive/My Drive/DBA Lab Test/DBALab-Test/Preprocessed/mHealth_subject10.csv',header=None)])

In [ ]:
predict = data[data[21]==0]

In [27]:
training = data[data[21]>0]

In [53]:
x.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
6656,-9.78,0.56,1.20,0.01,-0.03,0.38,-0.21,-0.89,-0.51,0.56,0.54,-0.74,-2.84,-9.06,1.82,-0.06,-0.93,-0.34,0.36,-0.37,-0.35


In [33]:
#split data into training and test set
x = training[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]]
y = training[21]
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.1, 
                                                    random_state=42)

In [36]:
def create_clients(image_list, label_list, num_clients=10, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))}

In [37]:
#create clients
clients = create_clients(X_train, y_train, num_clients=10, initial='client')

In [54]:
clients

{'client_1': [(9, 6), (10, 5)],
 'client_10': [(7, 6), (16, 1)],
 'client_2': [(14, 1), (8, 1)],
 'client_3': [(20, 1), (15, 6)],
 'client_4': [(13, 4), (6, 5)],
 'client_5': [(18, 4), (1, 5)],
 'client_6': [(0, 6), (19, 3)],
 'client_7': [(12, 3), (3, 4)],
 'client_8': [(17, 2), (2, 3)],
 'client_9': [(5, 5), (11, 5)]}

In [48]:
def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model
    

In [42]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

In [43]:
clients_batched

{'client_1': <BatchDataset shapes: ((None,), (None,)), types: (tf.int32, tf.int32)>,
 'client_10': <BatchDataset shapes: ((None,), (None,)), types: (tf.int32, tf.int32)>,
 'client_2': <BatchDataset shapes: ((None,), (None,)), types: (tf.int32, tf.int32)>,
 'client_3': <BatchDataset shapes: ((None,), (None,)), types: (tf.int32, tf.int32)>,
 'client_4': <BatchDataset shapes: ((None,), (None,)), types: (tf.int32, tf.int32)>,
 'client_5': <BatchDataset shapes: ((None,), (None,)), types: (tf.int32, tf.int32)>,
 'client_6': <BatchDataset shapes: ((None,), (None,)), types: (tf.int32, tf.int32)>,
 'client_7': <BatchDataset shapes: ((None,), (None,)), types: (tf.int32, tf.int32)>,
 'client_8': <BatchDataset shapes: ((None,), (None,)), types: (tf.int32, tf.int32)>,
 'client_9': <BatchDataset shapes: ((None,), (None,)), types: (tf.int32, tf.int32)>}

In [44]:
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

comms_round = 100

In [51]:
#create optimizer
lr = 0.01 
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               ) 

#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(21, 6)
        
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(21, 6)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
      global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)
      SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(y_train)).batch(320)

smlp_SGD = SimpleMLP()
SGD_model = smlp_SGD.build(784, 10) 

SGD_model.compile(loss=loss, 
              optimizer=optimizer, 
              metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

#test the SGD global model and print out metrics
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)

ValueError: ignored

In [3]:
pip install --upgrade tensorflow_federated

     |████████████████████████████████| 522kB 3.4MB/s 
     |████████████████████████████████| 1.1MB 19.3MB/s 
     |████████████████████████████████| 3.0MB 19.3MB/s 
     |████████████████████████████████| 112kB 45.4MB/s 
     |████████████████████████████████| 153kB 43.4MB/s 
     |████████████████████████████████| 174kB 34.6MB/s 
  Created wheel for absl-py: filename=absl_py-0.9.0-cp36-none-any.whl size=121933 sha256=247ed4158aa1d2a9f862bc69a96507ec288fa1c2b8ddc0fd00c6f9fed62a2975
  Stored in directory: /root/.cache/pip/wheels/8e/28/49/fad4e7f0b9a1227708cbbee4487ac8558a7334849cb81c813d
Successfully built absl-py
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: tensorflow-addons 0.8.3
    Uninstalling tensorflow-addons-0.8.3:
      Successfully uninstalled tensorflow-addons-0.8.3
  Found existing installation: cachetools 4.1.1
    Uninstalling cachetools-4.1.1:
      Successfully uninstalled ca

In [1]:
import tensorflow as tf
import tensorflow_federated as tff

# Load simulation data.
source, _ = tff.simulation.datasets.emnist.load_data()
def client_data(n):
  return source.create_tf_dataset_for_client(source.client_ids[n]).map(
      lambda e: (tf.reshape(e['pixels'], [-1]), e['label'])
  ).repeat(10).batch(20)

97402880/97398400 [==============================] - 2s 0us/step


In [2]:
source

In [3]:
# Pick a subset of client devices to participate in training.
train_data = [client_data(n) for n in range(3)]

In [5]:
client_data(1)

<BatchDataset shapes: ((None, 784), (None,)), types: (tf.float32, tf.int32)>

In [4]:
train_data

[<BatchDataset shapes: ((None, 784), (None,)), types: (tf.float32, tf.int32)>,
 <BatchDataset shapes: ((None, 784), (None,)), types: (tf.float32, tf.int32)>,
 <BatchDataset shapes: ((None, 784), (None,)), types: (tf.float32, tf.int32)>]

In [6]:
# Wrap a Keras model for use with TFF.
def model_fn():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(10, tf.nn.softmax, input_shape=(784,),
                            kernel_initializer='zeros')
  ])
  return tff.learning.from_keras_model(
      model,
      input_spec=train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [7]:
# Simulate a few rounds of training with the selected client devices.
trainer = tff.learning.build_federated_averaging_process(model_fn,client_optimizer_fn=lambda: tf.keras.optimizers.SGD(0.1))

In [8]:
pip install nest_asyncio

In [9]:
import nest_asyncio
nest_asyncio.apply()

In [10]:
trainer

In [20]:
state = trainer.initialize()
for _ in range(5):
  state, metrics = trainer.next(state, train_data)
  print (metrics['train']['loss'])

11.406281
11.931888
10.773072
10.509466
8.917007
